# Random Forest
A Random Forest Classifier is a decision based ensemble method that uses a series of decision trees to determine the class of our input variables. It works by selecting a random set of features at each node then averaging them together. This helps reduce overfitting by not emphasizing any one particular feature over another. We felt this was our next best choice after Logistic Regression because decision trees make no assumptions about the spread of the data and are very quick to train. Though this was not our production model we did spend a good amount of time searching through hyperparameters to find an optimum fit. Our results with this modelexhibited overfitting with a low amount of
predictive power on unseen data.


In [1]:
import numpy as np
import pandas as pd
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.ensemble import RandomForestClassifier

# Importing the Time Series split assets

In [2]:
X_train = pd.read_pickle('../assets/X_train.pkl')
X_test = pd.read_pickle('../assets/X_test.pkl')
y_train = pd.read_pickle('../assets/y_train.pkl')
y_test = pd.read_pickle('../assets/y_test.pkl')


In [3]:
X_train.columns

Index(['Day_length', 'Tmax', 'Tmin', 'Tavg', 'ResultSpeed', 'ResultDir',
       'AvgSpeed', 'Sunset', 'Sunrise', 'Heat', 'Depart', 'DewPoint',
       'WetBulb', 'Cool', 'PrecipTotal', 'StnPressure', 'Latitude',
       'Longitude', 'Month', 'Day_length_shift', 'Tavg_shift', 'Heat_shift',
       'Cool_shift', 'Tmax_shift', 'Tmin_shift', 'Depart_shift',
       'ResultSpeed_shift', 'ResultDir_shift', 'WetBulb_shift',
       'Species_CULEX ERRATICUS', 'Species_CULEX PIPIENS',
       'Species_CULEX PIPIENS/RESTUANS', 'Species_CULEX RESTUANS',
       'Species_CULEX SALINARIUS', 'Species_CULEX TARSALIS',
       'Species_CULEX TERRITANS'],
      dtype='object')

In [4]:
X_train.drop(columns=['Day_length', 'Tmax', 'Tmin', 'Tavg', 'ResultSpeed', 'ResultDir',
       'AvgSpeed', 'Sunset', 'Sunrise', 'Heat', 'Depart', 'DewPoint',
       'WetBulb', 'Cool', 'PrecipTotal', 'StnPressure',],inplace=True)

In [5]:
X_test.drop(columns=['Day_length', 'Tmax', 'Tmin', 'Tavg', 'ResultSpeed', 'ResultDir',
       'AvgSpeed', 'Sunset', 'Sunrise', 'Heat', 'Depart', 'DewPoint',
       'WetBulb', 'Cool', 'PrecipTotal', 'StnPressure',],inplace=True)

In [6]:
X_train.columns

Index(['Latitude', 'Longitude', 'Month', 'Day_length_shift', 'Tavg_shift',
       'Heat_shift', 'Cool_shift', 'Tmax_shift', 'Tmin_shift', 'Depart_shift',
       'ResultSpeed_shift', 'ResultDir_shift', 'WetBulb_shift',
       'Species_CULEX ERRATICUS', 'Species_CULEX PIPIENS',
       'Species_CULEX PIPIENS/RESTUANS', 'Species_CULEX RESTUANS',
       'Species_CULEX SALINARIUS', 'Species_CULEX TARSALIS',
       'Species_CULEX TERRITANS'],
      dtype='object')

# Running a pipeline
With a standard scaler and a Random Forest Classifier

In [7]:
pipe = Pipeline([
    ('ss',StandardScaler()),
    ('rfc',RandomForestClassifier(n_jobs=3,n_estimators=100,random_state=42))   
])

# Setting the range of hyperparameters 

In [8]:
param_grid =  {
    'rfc__min_samples_split': [2, 7, 10, 20]
        
}

# Gridsearching with  timeseries split
We used roc_auc as our scoring metric paired with a timeseries split.

In [9]:
gs = GridSearchCV(pipe, param_grid=param_grid,verbose=1,scoring='roc_auc', cv=TimeSeriesSplit())

In [ ]:
gs.fit(X_train,y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


# Scoring
Our data was routinely overfit with this model. 

In [ ]:
gs.score(X_train,y_train)

In [ ]:
gs.score(X_test,y_test)

In [ ]:
X_train_preds = gs.predict(X_train)

In [ ]:
preds = pd.DataFrame({
    "preds":X_train_preds,
    "truth":y_train
})


In [ ]:
preds.sum()

In [ ]:
gs.best_params_

In [ ]:
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit

# Feature importances
The first 7 features account for most of the information in our model.

In [ ]:
feat_importances = pd.DataFrame(gs.best_estimator_.named_steps['rfc'].feature_importances_, X_train.columns, columns=['importance'])

In [ ]:
feat_importances.sort_values('importance', ascending=False)

In [ ]:
with open('../assets/rf_0925_303.pkl','wb+') as f:
    pickle.dump(gs,f)